In [1]:
import os
import pandas as pd
import numpy as np
import xgboost
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score, roc_auc_score, roc_curve
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
import matplotlib.pyplot as plt
import seaborn as sns
import shap

In [2]:
csv_dir = '../SEC Trackman Data/'
all_files = [os.path.join(csv_dir, f) for f in os.listdir(csv_dir) if f.endswith('.csv')]

df_list = []

for file in all_files:
    df = pd.read_csv(file)
    df_list.append(df)

all_pitches = pd.concat(df_list, ignore_index=True)

print(all_pitches.shape)
print(all_pitches.columns)

(270589, 167)
Index(['PitchNo', 'Date', 'Time', 'PAofInning', 'PitchofPA', 'Pitcher',
       'PitcherId', 'PitcherThrows', 'PitcherTeam', 'Batter',
       ...
       'ThrowTrajectoryZc1', 'ThrowTrajectoryZc2', 'PitchReleaseConfidence',
       'PitchLocationConfidence', 'PitchMovementConfidence',
       'HitLaunchConfidence', 'HitLandingConfidence',
       'CatcherThrowCatchConfidence', 'CatcherThrowReleaseConfidence',
       'CatcherThrowLocationConfidence'],
      dtype='object', length=167)


In [3]:
fastballs = all_pitches[all_pitches['TaggedPitchType'].isin(['Fastball','Cutter','Sinker','FourSeamFastBall','TwoSeamFastBall','OneSeamFastBall'])]

In [4]:
fastballs['whiff'] = np.where(fastballs['PitchCall']=='StrikeSwinging',1,0)

C:\Users\charl\AppData\Local\Temp\ipykernel_23324\409369648.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fastballs['whiff'] = np.where(fastballs['PitchCall']=='StrikeSwinging',1,0)


In [5]:
preds = ['RelSpeed','VertRelAngle','HorzRelAngle','SpinRate','SpinAxis','RelHeight','RelSide','Extension','VertBreak','InducedVertBreak','HorzBreak','ZoneSpeed','VertApprAngle','HorzApprAngle','ZoneTime','pfxx','pfxz','x0','y0','z0','vx0','vy0','vz0','ax0','ay0','az0','SpeedDrop','PitchTrajectoryXc1','PitchTrajectoryXc2','PitchTrajectoryYc0','PitchTrajectoryYc1','PitchTrajectoryYc2','PitchTrajectoryZc0','PitchTrajectoryZc1','PitchTrajectoryZc2']

In [6]:
fastballs_X = fastballs[preds]
fastballs_y = fastballs['whiff']

In [7]:
X_train, X_test, y_train, y_test = train_test_split(fastballs_X, fastballs_y, test_size=.33)

model = XGBClassifier()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]

accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 90.76%


In [9]:
# Get unique values
unique_values = fastballs['Pitcher'].unique()

# Create a dictionary to store subsets
subsets = {value: fastballs[fastballs['Pitcher'] == value] for value in unique_values}

In [10]:
all_preds = model.predict_proba(fastballs_X)[:,1]
mean_pred = np.mean(all_preds)

t_bauman = subsets['Bauman, Tanner']
t_bauman_X = t_bauman[preds]

whiff_preds = model.predict_proba(t_bauman_X)[:,1]

stuff = whiff_preds/mean_pred

print(f"Tanner Bauman fastball stuff+:{np.mean(stuff*100)}")

Tanner Bauman fastball stuff+:80.46209716796875


In [24]:
p_skenes = subsets['Causey, A.J.']
p_skenes_X = p_skenes[preds]
whiff_preds = model.predict_proba(p_skenes_X)[:,1]
stuff = whiff_preds/mean_pred

print(np.mean(stuff*100))

54.39105


In [11]:
tb_spd_avg = np.mean(t_bauman['RelSpeed'])
tb_spd_std = np.std(t_bauman['RelSpeed'])

tb_hgt_avg = np.mean(t_bauman['RelHeight'])
tb_hgt_std = np.std(t_bauman['RelHeight'])

tb_side_avg = np.mean(t_bauman['RelSide'])
tb_side_std = np.std(t_bauman['RelSide'])

In [12]:
similar_fastballs = fastballs[(fastballs['RelSpeed'] >= tb_spd_avg - tb_spd_std) &
                              (fastballs['RelSpeed'] <= tb_spd_avg + tb_spd_std) &
                              (fastballs['RelHeight'] >= tb_hgt_avg - tb_hgt_std) &
                              (fastballs['RelHeight'] <= tb_hgt_avg + tb_hgt_std) &
                              (fastballs['RelSide'] >= tb_side_avg - tb_side_std) &
                              (fastballs['RelSide'] <= tb_side_avg + tb_side_std)]

print(similar_fastballs.shape)

(1746, 168)


In [20]:
similar_fastballs['stuff_plus'] = (model.predict_proba(similar_fastballs[preds])[:,1])/mean_pred * 100

print(f"Average Stuff+ of similar Fastballs:{np.mean(similar_fastballs['stuff_plus'])}")

Average Stuff+ of similar Fastballs:103.71044158935547


C:\Users\charl\AppData\Local\Temp\ipykernel_23324\1472775353.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  similar_fastballs['stuff_plus'] = (model.predict_proba(similar_fastballs[preds])[:,1])/mean_pred * 100
